In [14]:
import torch
import torch.nn as nn
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [15]:
x_train = pd.read_csv('interim/x_train.csv')
y_train = pd.read_csv('interim/y_train.csv')
y_train_month = y_train['target_month']
y_train_day = y_train['target_day']
x_test = pd.read_csv('interim/x_test.csv')

# convert to tensors
x_train = torch.tensor(x_train.values, dtype=torch.float)
y_train_month = torch.tensor(y_train_month.values, dtype=torch.float)
y_train_day = torch.tensor(y_train_day.values, dtype=torch.float)
x_test = torch.tensor(x_test.values, dtype=torch.float)

In [16]:
from torch.utils.data import Dataset

class MonthDayDataset(Dataset):
    def __init__(self, x, y_month=None, y_day=None):
        self.x = x
        self.train = y_month is not None and y_day is not None
        if self.train:
            self.y_month = y_month.reshape(-1, 1)
            self.y_day = y_day.reshape(-1, 1)
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        if self.train:
            return self.x[idx], self.y_month[idx], self.y_day[idx]
        else:
            return self.x[idx]

In [17]:
from torch.utils.data import DataLoader, random_split
dataset = MonthDayDataset(x_train, y_train_month, y_train_day)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True)

In [18]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(50, 128)
        self.fc2 = nn.Linear(128, 256)
        self.fc3 = nn.Linear(256, 512)
        self.fc4 = nn.Linear(512, 256)
        self.fc5 = nn.Linear(256, 128)
        self.fc6 = nn.Linear(128, 64)
        self.fc7 = nn.Linear(64, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.relu(self.fc4(x))
        x = self.dropout(x)
        x = self.relu(self.fc5(x))
        x = self.dropout(x)
        x = self.relu(self.fc6(x))
        x = self.dropout(x)
        x = self.sigmoid(self.fc7(x))
        return x

month_model = Net()
day_model = Net()

In [19]:
from torch.optim import Adam
from torch.nn import BCELoss

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

criterion_month = BCELoss()
criterion_day = BCELoss()
optimizer_month = Adam(month_model.parameters(), lr=1e-3)
optimizer_day = Adam(day_model.parameters(), lr=1e-3)

In [20]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
scheduler_month = ReduceLROnPlateau(optimizer_month, 'min', patience=5)
scheduler_day = ReduceLROnPlateau(optimizer_day, 'min', patience=5)

In [24]:
from tqdm import tqdm
from sklearn.metrics import f1_score
import numpy as np

def get_f1_score(y_month, y_day, target_month, target_day):
    y_month = (y_month.cpu().detach().numpy() > 0.05).astype(int)
    y_day = (y_day.cpu().detach().numpy() > 0.02).astype(int)
    target_month = target_month.cpu().detach().numpy()
    target_day = target_day.cpu().detach().numpy()
    return 0.5 * f1_score(target_month, y_month) + 0.5 * f1_score(target_day, y_day) 

def train_and_eval(epoch):
    month_model.train()
    day_model.train()
    pbar = tqdm(train_loader)
    f1s = []
    losses = []
    for data, target_month, target_day in pbar:
        data, target_month, target_day = data.to(device), target_month.to(device), target_day.to(device)
        optimizer_month.zero_grad()
        optimizer_day.zero_grad()
        output_month = month_model(data)
        output_day = day_model(data)
        loss_month = criterion_month(output_month, target_month)
        loss_day = criterion_day(output_day, target_day)
        loss = loss_month + loss_day
        losses.append(loss.item())
        loss.backward()
        optimizer_month.step()
        optimizer_day.step()
        f1s.append(get_f1_score(output_month, output_day, target_month, target_day))
        pbar.set_description(f'Epoch {epoch} Loss: {np.mean(losses):.5f} F1: {np.mean(f1s):.5f}')

    month_model.eval()
    day_model.eval()
    pbar = tqdm(val_loader)
    f1s = []
    losses = []
    for data, target_month, target_day in pbar:
        data, target_month, target_day = data.to(device), target_month.to(device), target_day.to(device)
        output_month = month_model(data)
        output_day = day_model(data)
        loss_month = criterion_month(output_month, target_month)
        loss_day = criterion_day(output_day, target_day)
        loss = loss_month + loss_day
        losses.append(loss.item())
        f1s.append(get_f1_score(output_month, output_day, target_month, target_day))
        pbar.set_description(f'Validation Epoch {epoch} Loss: {np.mean(losses):.5f} F1: {np.mean(f1s):.5f}')

    return np.mean(losses), np.mean(f1s)

In [25]:
epochs = 10
month_model.to(device)
day_model.to(device)

best_loss = 1e9

for epoch in range(epochs):
    loss, f1 = train_and_eval(epoch)
    scheduler_month.step(f1)
    scheduler_day.step(f1)
    if loss < best_loss:
        best_loss = loss
        torch.save(month_model.state_dict(), 'models/month_model.pt')
        torch.save(day_model.state_dict(), 'models/day_model.pt')

Validation Epoch 9 Loss: 0.22680 F1: 0.13936: 100%|██████████| 638/638 [00:07<00:00, 80.09it/s]


In [26]:
tmp_target_path = './prediction/target_predicton_tmp.csv'
target_path = './prediction/target_predicton.csv'

In [28]:
test_dataset = MonthDayDataset(x_test)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

month_model.eval()
day_model.eval()
y_month = []
y_day = []
for data in tqdm(test_loader):
    data = data.to(device)
    output_month = month_model(data)
    output_day = day_model(data)
    y_month.append((output_month.cpu().detach().numpy() > 0.05).astype(int))
    y_day.append((output_day.cpu().detach().numpy() > 0.02).astype(int))

y_month = np.concatenate(y_month).reshape(-1)
y_day = np.concatenate(y_day).reshape(-1)

for i in range(len(y_month)):
    if y_day[i] == 1:
        y_month[i] = 1

predict_data = pd.read_csv('target/y_predict.csv')
preds_df = pd.DataFrame({'target_month': y_month, 'target_day': y_day}, columns=['target_month', 'target_day'])
preds_df = pd.concat([predict_data, preds_df], axis=1)
preds_df = preds_df.drop('month', axis=1)
preds_df = preds_df.sort_values(by=['wagnum'])
preds_df.to_csv(target_path, index=False)
print(len(preds_df))

100%|██████████| 527/527 [00:02<00:00, 236.29it/s]


33707


In [29]:
from metrics_f1 import calc_f1_score
calc_f1_score( tmp_target_path, target_path,)

0.05626477569676727